In [148]:
import pandas as pd
import numpy as np
import collections
import operator
import re

In [ ]:
## Here goes the implementation for predicing haplogroups

In [68]:
df_intermediate = pd.read_csv("Hg_Prediction_tables/Intermediates.txt", header=None)
intermediates = df_intermediate[0].values

In [51]:
df_haplogroup = pd.read_csv("Output_example_datasets/Modern_DNA_WGS/HG003/HG003.out", sep="\t")

In [52]:
df_haplogroup = df_haplogroup.sort_values(by=['haplogroup'])

In [53]:
df_intermediate

array(['A00-T', 'A000-T', 'A0-T', 'A1', 'A1b', 'BT', 'CT', 'CF', 'DE',
       'F', 'GHIJK', 'HIJK', 'IJK', 'IJ', 'NO', 'NO1', 'LT', 'K', 'K~',
       'K2', 'P', 'P~', 'P1', 'P1~'], dtype=object)

### Derive State

#### Removes the intermediate haplogroups temporally

In [54]:
df_derived = df_haplogroup[df_haplogroup["state"] == "D"]

In [181]:
## instance with only D state
df_tmp = df_derived
for hg in intermediates:    
    df_tmp = df_tmp.drop(df_tmp[df_tmp.haplogroup == hg].index)
hg = df_tmp["haplogroup"].values

#### Choose the haplogroup based on the highest count in the first letter (E.x. J2a21 = J)

In [182]:
list_hg = []
for i in hg:
    list_hg.append(i[0])
collections_hg = collections.Counter(list_hg)

In [208]:
init_hg = max(collections_hg.items(), key=operator.itemgetter(1))[0]
init_hg = init_hg+"_int.txt"
hg_intermediate_file = "Hg_Prediction_tables/"+init_hg
df_intermediate = pd.read_csv(hg_intermediate_file, header=None, sep="\t")
#df_intermediate = df_intermediate.values

### QC.1 
Now you have the putative haplogroup. Open the file with the haplogroup intermediate and show the intermediate haplogroups again and compare if there are some where does not match with the correct state and store it a error. Give the estimate of the  correct_count/total  

$$QC1 = \frac{correct state } {total}$$

where 
###### correct state is the total number of correct haplogroup state found in the intermediate haplogroup file
###### total of all intermediate haplogroup found in the intermediate haplogroup file

In [219]:
print(df_intermediate)
total = 0
for i in df_intermediate:
    print(df_intermediate[i])
    break
    #df_tmp = df_derived.loc[df_derived["haplogroup"] == i]
    #total += df_tmp.shape[0]
    #if total > 0:
    #    print(df_tmp)
    #    print(df_intermediate[0] == )
    #    break
print(total)


         0  1
0    A00-T  D
1   A000-T  D
2     A0-T  D
3       A1  D
4      A1b  D
5       BT  D
6       CT  D
7       CF  D
8       DE  A
9        F  D
10   GHIJK  D
11    HIJK  D
12     IJK  D
13      IJ  D
14      NO  A
15     NO1  A
16      LT  A
17       K  A
18      K~  A
19      K2  A
20       P  A
21      P~  A
22      P1  A
23     P1~  A
0      A00-T
1     A000-T
2       A0-T
3         A1
4        A1b
5         BT
6         CT
7         CF
8         DE
9          F
10     GHIJK
11      HIJK
12       IJK
13        IJ
14        NO
15       NO1
16        LT
17         K
18        K~
19        K2
20         P
21        P~
22        P1
23       P1~
Name: 0, dtype: object
0


Removes all haplogroup but the main one
Check if the preffix of all main haplogroup with D state by allowing one haplogroup that does not match to the main haplogroup until you get back to the origin. 
If mismatch > 1 skip choose as main haplogroup the following in the largest length and continue. In case haplogroup contains ~ symbol at the end ignore it temporally for preffix comparison but if this is the main haplogroup store the special character at the end. 

### Ancenstral and Derive State

### QC.2

Check if the same name of the main haplogroup appears as an Ancestral State and save the number of count and calculate QC2:

$$QC2 = \frac{Dhg-Ahg} {total}$$
where 
###### Dhg is the total number of derived main haplogroup 
###### Ahg is the total number of ancestral with same name of main haplogroup
###### total of all derived and ancestral haplogroup with the same name

### QC.3
Show both Ancestral and Derived states from the main haplogroup and check the preffix of each haplogroup that is before the main one until you reach the root. If there are some Ancestral states which follows the pattern/preffix from the main haplogroup keep the count of how many of these appear and this will give the total count. Substract the ones found from the corrected and this will give the QC.3 score. 

### Final output
1) show the haplogroup name 
    if main haplogroup is a preffix from a higher resolution Ancestral state haplogroup report this haplogroup and marker name (Ex. HG: J21a Marker: L123)
    
2) Not J21a-L123

Could be that that there are more than one contain as a preffix from an ancestral state haplogroup. Should report all of them only if there are different haplogroup name with the resolution
## Check this more in detail later on... 